<a href="https://colab.research.google.com/github/comaria23/CNN-Brain-TumorDetection/blob/main/Saved_model_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
import ipywidgets as widgets
import io
from PIL import Image
from IPython.display import display,clear_output
from warnings import filterwarnings

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
labels = ['glioma','notumor','meningioma','pituitary']

In [4]:
X_train = []
y_train = []
X_test=[]
y_test=[]
image_size = 224
for i in labels:
    folderPath = os.path.join('/content/gdrive/My Drive/Date_disertatie','Training',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)

for i in labels:
    folderPath = os.path.join('/content/gdrive/My Drive/Date_disertatie','Testing',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_test.append(img)
        y_test.append(i)

X_train = np.array(X_train)
X_test= np.array(X_test)

100%|██████████| 300/300 [00:09<00:00, 32.23it/s]


In [5]:
y_train_ = []
for i in y_train:
    y_train_.append(labels.index(i))
Y_train = y_train_

Y_train = tf.keras.utils.to_categorical(Y_train)

y_test_ = []
for i in y_test:
    y_test_.append(labels.index(i))
Y_test = y_test_

Y_test = tf.keras.utils.to_categorical(Y_test)

In [6]:
new_model = tf.keras.models.load_model('/content/gdrive/My Drive/Colab Notebooks/brain_tumor_classification.h5')

In [ ]:
pred = new_model.predict(X_test)
pred = np.argmax(pred,axis=1)
y_test_new = np.argmax(Y_test,axis=1)

29/41 [====================>.........] - ETA: 27s

In [ ]:
def img_pred(upload):
    for name, file_info in uploader.value.items():
        img = Image.open(io.BytesIO(file_info['content']))
    opencvImage = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    img = cv2.resize(opencvImage,(224,224))
    img = img.reshape(1,224,224,3)
    p = new_model.predict(img)
    p = np.argmax(p,axis=1)[0]

    if p==0:
        p='Glioma'
    elif p==1:
        p='No Tumor'
    elif p==2:
        print('Meningioma')
    else:
        p='Pituitary Tumor'

    if p!=2:
        print(f'Prediction: {p}')

In [ ]:
uploader = widgets.FileUpload()
display(uploader)
button = widgets.Button(description='Prediction')
out = widgets.Output()
def on_button_clicked(_):
    with out:
        clear_output()
        try:
            img_pred(uploader)
        except:
            print('Error')
button.on_click(on_button_clicked)
widgets.VBox([button,out])

FileUpload(value={}, description='Upload')

In [ ]:
test_pred = new_model.predict(X_test)
test_pred = np.argmax(test_pred, axis=1)
Y_test_ = np.argmax(Y_test, axis=1)
print("Acuracy for test set: {:.2f}%".format(np.sum(test_pred==Y_test_)/len(Y_test_)*100))

Acuratetea pe setul de testare: 98.86%


In [ ]:
train_pred = new_model.predict(X_train)
train_pred = np.argmax(train_pred, axis=1)
Y_train_new = np.argmax(Y_train, axis=1)
print("Acuracy for training set: {:.2f}%".format(np.sum(train_pred==Y_train_new)/len(Y_train_new)*100))

Acuratetea pentru setul de antrenare: 99.63%
